# 🚀 Code Translator from Python to C++

**Multi-LLM Python to C++ Code Translator with Compilation Testing and Quality Analysis**

This notebook demonstrates a comprehensive AI-powered code translation system that:
- Translates Python code to C++ using multiple LLM models (GPT-4o, Claude 3.5 Sonnet, Gemini 2.0 Flash)
- Automatically compiles and tests generated C++ code
- Performs quality analysis and performance benchmarking
- Compares translation results across different AI models

## 🎯 Key Features

- **Multi-LLM Support**: Compare translations from OpenAI, Anthropic, and Google
- **C++ Compilation**: Automatic compilation and execution testing
- **Quality Analysis**: Code quality metrics and performance benchmarking
- **Interactive Interface**: Easy-to-use notebook interface
- **Comprehensive Testing**: Full test suite for validation

## 📋 Table of Contents

1. [Setup and Installation](#setup)
2. [LLM Client Implementation](#llm-clients)
3. [C++ Compiler and Testing](#compiler)
4. [Core Translation Logic](#translator)
5. [Quality Analysis](#quality)
6. [Interactive Examples](#examples)
7. [Performance Benchmarking](#benchmarking)
8. [Testing and Validation](#testing)


## 1. Setup and Installation

First, let's install the required dependencies and set up the environment.


In [ ]:
# Install required packages
!uv add openai anthropic google-generativeai gradio python-dotenv pydantic requests psutil memory-profiler pytest black flake8 mypy
#For those working with pip, you can use the following command:
#!pip install openai anthropic google-generativeai gradio python-dotenv pydantic requests psutil memory-profiler pytest black flake8 mypy


In [ ]:
# Import required libraries
import os
import sys
import json
import time
import subprocess
import tempfile
import psutil
import re
from typing import Dict, List, Optional, Tuple, Any, Union
from dataclasses import dataclass, asdict
from pathlib import Path

# LLM libraries
import openai
import anthropic
import google.generativeai as genai
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

print("✅ All libraries imported successfully!")
print(f"Python version: {sys.version}")
print(f"Working directory: {os.getcwd()}")


## 2. LLM Client Implementation

Let's implement the LLM clients for OpenAI GPT, Anthropic Claude, and Google Gemini.


In [ ]:
# Data classes for translation results
@dataclass
class TranslationResult:
    """Result of a code translation."""
    source_code: str
    translated_code: str
    model_name: str
    success: bool
    error_message: Optional[str] = None
    translation_time: float = 0.0
    token_usage: Optional[Dict] = None

@dataclass
class CompilationResult:
    """Result of C++ compilation."""
    success: bool
    executable_path: Optional[str] = None
    error_message: Optional[str] = None
    compilation_time: float = 0.0
    warnings: List[str] = None

@dataclass
class ExecutionResult:
    """Result of C++ code execution."""
    success: bool
    output: str = ""
    error_message: Optional[str] = None
    execution_time: float = 0.0
    memory_usage: float = 0.0
    exit_code: int = 0

@dataclass
class PerformanceMetrics:
    """Performance metrics for C++ code."""
    execution_time: float
    memory_usage: float
    cpu_usage: float
    code_size: int
    compilation_time: float

print("✅ Data classes defined successfully!")


In [ ]:
# OpenAI GPT Client
class OpenAIClient:
    """OpenAI GPT client for code translation."""
    
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.client = openai.OpenAI(api_key=api_key)
    
    def translate_python_to_cpp(self, python_code: str, context: str = "") -> TranslationResult:
        """Translate Python code to C++ using GPT-4o."""
        start_time = time.time()
        
        try:
            system_prompt = """You are an expert Python to C++ translator. 
            Convert the given Python code to efficient, modern C++ code.
            
            Requirements:
            - Use modern C++17/20 features
            - Include proper headers
            - Add comprehensive error handling
            - Optimize for performance
            - Include detailed comments
            - Follow C++ best practices
            
            Return ONLY the C++ code, no explanations."""
            
            user_prompt = f"""Translate this Python code to C++:

Context: {context}

Python Code:
```python
{python_code}
```

C++ Translation:"""
            
            response = self.client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0.1,
                max_tokens=4000
            )
            
            translated_code = response.choices[0].message.content.strip()
            translation_time = time.time() - start_time
            
            return TranslationResult(
                source_code=python_code,
                translated_code=translated_code,
                model_name="GPT-4o",
                success=True,
                translation_time=translation_time,
                token_usage={
                    "prompt_tokens": response.usage.prompt_tokens,
                    "completion_tokens": response.usage.completion_tokens,
                    "total_tokens": response.usage.total_tokens
                }
            )
            
        except Exception as e:
            return TranslationResult(
                source_code=python_code,
                translated_code="",
                model_name="GPT-4o",
                success=False,
                error_message=str(e),
                translation_time=time.time() - start_time
            )

print("✅ OpenAI client implemented!")


In [ ]:
# Anthropic Claude Client
class ClaudeClient:
    """Anthropic Claude client for code translation."""
    
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.client = anthropic.Anthropic(api_key=api_key)
    
    def translate_python_to_cpp(self, python_code: str, context: str = "") -> TranslationResult:
        """Translate Python code to C++ using Claude 3.5 Sonnet."""
        start_time = time.time()
        
        try:
            system_prompt = """You are an expert Python to C++ translator. 
            Convert the given Python code to efficient, modern C++ code.
            
            Requirements:
            - Use modern C++17/20 features
            - Include proper headers
            - Add comprehensive error handling
            - Optimize for performance
            - Include detailed comments
            - Follow C++ best practices
            
            Return ONLY the C++ code, no explanations."""
            
            user_prompt = f"""Translate this Python code to C++:

Context: {context}

Python Code:
```python
{python_code}
```

C++ Translation:"""
            
            response = self.client.messages.create(
                model="claude-sonnet-4-20250514",
                max_tokens=4000,
                temperature=0.1,
                system=system_prompt,
                messages=[
                    {"role": "user", "content": user_prompt}
                ]
            )
            
            translated_code = response.content[0].text.strip()
            translation_time = time.time() - start_time
            
            return TranslationResult(
                source_code=python_code,
                translated_code=translated_code,
                model_name="Claude-3.5-Sonnet",
                success=True,
                translation_time=translation_time,
                token_usage={
                    "input_tokens": response.usage.input_tokens,
                    "output_tokens": response.usage.output_tokens
                }
            )
            
        except Exception as e:
            return TranslationResult(
                source_code=python_code,
                translated_code="",
                model_name="Claude-3.5-Sonnet",
                success=False,
                error_message=str(e),
                translation_time=time.time() - start_time
            )

print("✅ Claude client implemented!")


In [ ]:
# Google Gemini Client
class GeminiClient:
    """Google Gemini client for code translation."""
    
    def __init__(self, api_key: str):
        self.api_key = api_key
        genai.configure(api_key=api_key)
        self.client = genai.GenerativeModel('gemini-2.0-flash-exp')
    
    def translate_python_to_cpp(self, python_code: str, context: str = "") -> TranslationResult:
        """Translate Python code to C++ using Gemini 2.0 Flash."""
        start_time = time.time()
        
        try:
            prompt = f"""You are an expert Python to C++ translator. 
            Convert the given Python code to efficient, modern C++ code.
            
            Requirements:
            - Use modern C++17/20 features
            - Include proper headers
            - Add comprehensive error handling
            - Optimize for performance
            - Include detailed comments
            - Follow C++ best practices
            
            Context: {context}
            
            Python Code:
            ```python
            {python_code}
            ```
            
            Return ONLY the C++ code, no explanations."""
            
            response = self.client.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    temperature=0.1,
                    max_output_tokens=4000
                )
            )
            
            translated_code = response.text.strip()
            translation_time = time.time() - start_time
            
            return TranslationResult(
                source_code=python_code,
                translated_code=translated_code,
                model_name="Gemini-2.0-Flash",
                success=True,
                translation_time=translation_time
            )
            
        except Exception as e:
            return TranslationResult(
                source_code=python_code,
                translated_code="",
                model_name="Gemini-2.0-Flash",
                success=False,
                error_message=str(e),
                translation_time=time.time() - start_time
            )

print("✅ Gemini client implemented!")


In [ ]:
# LLM Client Manager
class LLMClientManager:
    """Manages multiple LLM clients for code translation."""
    
    def __init__(self):
        self.clients = {}
        self._initialize_clients()
    
    def _initialize_clients(self):
        """Initialize available LLM clients."""
        # OpenAI
        openai_key = os.getenv('OPENAI_API_KEY')
        if openai_key:
            self.clients['gpt'] = OpenAIClient(openai_key)
        
        # Anthropic Claude
        claude_key = os.getenv('ANTHROPIC_API_KEY')
        if claude_key:
            self.clients['claude'] = ClaudeClient(claude_key)
        
        # Google Gemini
        gemini_key = os.getenv('GOOGLE_API_KEY')
        if gemini_key:
            self.clients['gemini'] = GeminiClient(gemini_key)
    
    def get_available_models(self) -> List[str]:
        """Get list of available model names."""
        return list(self.clients.keys())
    
    def translate_with_all_models(self, python_code: str, context: str = "") -> Dict[str, TranslationResult]:
        """Translate code using all available models."""
        results = {}
        
        for model_name, client in self.clients.items():
            try:
                result = client.translate_python_to_cpp(python_code, context)
                results[model_name] = result
            except Exception as e:
                results[model_name] = TranslationResult(
                    source_code=python_code,
                    translated_code="",
                    model_name=model_name,
                    success=False,
                    error_message=str(e)
                )
        
        return results
    
    def translate_with_model(self, model_name: str, python_code: str, context: str = "") -> TranslationResult:
        """Translate code using a specific model."""
        if model_name not in self.clients:
            raise ValueError(f"Model {model_name} not available. Available models: {list(self.clients.keys())}")
        
        return self.clients[model_name].translate_python_to_cpp(python_code, context)

# Initialize LLM manager
llm_manager = LLMClientManager()
available_models = llm_manager.get_available_models()

print(f"✅ LLM Client Manager initialized!")
print(f"Available models: {available_models}")

if not available_models:
    print("⚠️ No LLM models available. Please check your API keys:")
    print("  - OPENAI_API_KEY")
    print("  - ANTHROPIC_API_KEY") 
    print("  - GOOGLE_API_KEY")


## 3. C++ Compiler and Testing

Now let's implement the C++ compilation and testing functionality.


In [ ]:
# C++ Compiler Implementation
class CppCompiler:
    """Handles C++ compilation and testing."""
    
    def __init__(self, compiler_path: str = "g++", optimization_level: str = "-O2"):
        self.compiler_path = compiler_path
        self.optimization_level = optimization_level
        self.temp_dir = None
    
    def __enter__(self):
        """Context manager entry."""
        self.temp_dir = tempfile.mkdtemp(prefix="cpp_translator_")
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        """Context manager exit - cleanup temp files."""
        if self.temp_dir and os.path.exists(self.temp_dir):
            import shutil
            shutil.rmtree(self.temp_dir, ignore_errors=True)
    
    def _write_cpp_file(self, cpp_code: str, filename: str = "main.cpp") -> str:
        """Write C++ code to a temporary file."""
        if not self.temp_dir:
            raise RuntimeError("Compiler not initialized. Use as context manager.")
        
        file_path = os.path.join(self.temp_dir, filename)
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(cpp_code)
        return file_path
    
    def _add_standard_headers(self, cpp_code: str) -> str:
        """Add standard C++ headers if not present."""
        if "#include" not in cpp_code:
            headers = [
                "#include <iostream>",
                "#include <vector>",
                "#include <string>",
                "#include <algorithm>",
                "#include <memory>",
                "#include <stdexcept>",
                "#include <chrono>",
                "#include <thread>"
            ]
            cpp_code = "\n".join(headers) + "\n\n" + cpp_code
        
        return cpp_code
    
    def _add_main_function_if_needed(self, cpp_code: str) -> str:
        """Add main function if not present."""
        if "int main(" not in cpp_code and "void main(" not in cpp_code:
            main_code = """
int main() {
    try {
        // Your code will be executed here
        return 0;
    } catch (const std::exception& e) {
        std::cerr << "Error: " << e.what() << std::endl;
        return 1;
    }
}"""
            cpp_code += main_code
        
        return cpp_code
    
    def compile_cpp(self, cpp_code: str, output_name: str = "main") -> CompilationResult:
        """Compile C++ code to executable."""
        start_time = time.time()
        
        try:
            # Preprocess the code
            cpp_code = self._add_standard_headers(cpp_code)
            cpp_code = self._add_main_function_if_needed(cpp_code)
            
            # Write to temporary file
            cpp_file = self._write_cpp_file(cpp_code)
            exe_path = os.path.join(self.temp_dir, output_name)
            
            # Compilation command
            cmd = [
                self.compiler_path,
                self.optimization_level,
                "-std=c++17",
                "-Wall",
                "-Wextra",
                cpp_file,
                "-o", exe_path
            ]
            
            # Compile
            result = subprocess.run(
                cmd,
                capture_output=True,
                text=True,
                timeout=30
            )
            
            compilation_time = time.time() - start_time
            
            if result.returncode == 0:
                return CompilationResult(
                    success=True,
                    executable_path=exe_path,
                    compilation_time=compilation_time,
                    warnings=self._extract_warnings(result.stderr)
                )
            else:
                return CompilationResult(
                    success=False,
                    error_message=result.stderr,
                    compilation_time=compilation_time
                )
                
        except subprocess.TimeoutExpired:
            return CompilationResult(
                success=False,
                error_message="Compilation timeout",
                compilation_time=time.time() - start_time
            )
        except Exception as e:
            return CompilationResult(
                success=False,
                error_message=str(e),
                compilation_time=time.time() - start_time
            )
    
    def _extract_warnings(self, stderr: str) -> List[str]:
        """Extract warnings from compiler output."""
        warnings = []
        for line in stderr.split('\n'):
            if 'warning:' in line.lower():
                warnings.append(line.strip())
        return warnings
    
    def execute_cpp(self, executable_path: str, input_data: str = "", timeout: int = 10) -> ExecutionResult:
        """Execute compiled C++ code."""
        start_time = time.time()
        
        try:
            # Start process
            process = subprocess.Popen(
                [executable_path],
                stdin=subprocess.PIPE,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                text=True
            )
            
            # Monitor memory usage
            memory_usage = 0.0
            try:
                ps_process = psutil.Process(process.pid)
                memory_usage = ps_process.memory_info().rss / 1024 / 1024  # MB
            except (psutil.NoSuchProcess, psutil.AccessDenied):
                pass
            
            # Execute with timeout
            stdout, stderr = process.communicate(input=input_data, timeout=timeout)
            execution_time = time.time() - start_time
            
            return ExecutionResult(
                success=process.returncode == 0,
                output=stdout,
                error_message=stderr if stderr else None,
                execution_time=execution_time,
                memory_usage=memory_usage,
                exit_code=process.returncode
            )
            
        except subprocess.TimeoutExpired:
            process.kill()
            return ExecutionResult(
                success=False,
                error_message="Execution timeout",
                execution_time=time.time() - start_time
            )
        except Exception as e:
            return ExecutionResult(
                success=False,
                error_message=str(e),
                execution_time=time.time() - start_time
            )
    
    def compile_and_test(self, cpp_code: str, test_input: str = "") -> Tuple[CompilationResult, Optional[ExecutionResult]]:
        """Compile and test C++ code."""
        # Compile
        compilation_result = self.compile_cpp(cpp_code)
        
        if not compilation_result.success:
            return compilation_result, None
        
        # Execute
        execution_result = self.execute_cpp(compilation_result.executable_path, test_input)
        
        return compilation_result, execution_result

print("✅ C++ Compiler implemented!")


In [ ]:
# Code Quality Analyzer
class CodeQualityAnalyzer:
    """Analyzes code quality metrics."""
    
    @staticmethod
    def analyze_cpp_quality(cpp_code: str) -> Dict[str, Any]:
        """Analyze C++ code quality."""
        metrics = {
            "lines_of_code": len(cpp_code.split('\n')),
            "comment_ratio": CodeQualityAnalyzer._calculate_comment_ratio(cpp_code),
            "function_count": CodeQualityAnalyzer._count_functions(cpp_code),
            "class_count": CodeQualityAnalyzer._count_classes(cpp_code),
            "complexity_score": CodeQualityAnalyzer._calculate_complexity(cpp_code),
            "style_score": CodeQualityAnalyzer._calculate_style_score(cpp_code),
            "error_handling": CodeQualityAnalyzer._check_error_handling(cpp_code),
            "modern_cpp_features": CodeQualityAnalyzer._check_modern_features(cpp_code)
        }
        
        return metrics
    
    @staticmethod
    def _calculate_comment_ratio(cpp_code: str) -> float:
        """Calculate ratio of commented lines."""
        lines = cpp_code.split('\n')
        comment_lines = sum(1 for line in lines if line.strip().startswith('//') or line.strip().startswith('/*'))
        return comment_lines / len(lines) if lines else 0.0
    
    @staticmethod
    def _count_functions(cpp_code: str) -> int:
        """Count function definitions."""
        pattern = r'\w+\s+\w+\s*\([^)]*\)\s*\{'
        return len(re.findall(pattern, cpp_code))
    
    @staticmethod
    def _count_classes(cpp_code: str) -> int:
        """Count class definitions."""
        pattern = r'class\s+\w+'
        return len(re.findall(pattern, cpp_code))
    
    @staticmethod
    def _calculate_complexity(cpp_code: str) -> int:
        """Calculate cyclomatic complexity."""
        complexity_keywords = ['if', 'else', 'while', 'for', 'switch', 'case', 'catch', '&&', '||']
        complexity = 1  # Base complexity
        
        for keyword in complexity_keywords:
            complexity += cpp_code.count(keyword)
        
        return complexity
    
    @staticmethod
    def _calculate_style_score(cpp_code: str) -> float:
        """Calculate style score based on various factors."""
        score = 0.0
        lines = cpp_code.split('\n')
        
        # Check for consistent indentation
        if all(line.startswith((' ', '\t')) or not line.strip() for line in lines[1:]):
            score += 0.2
        
        # Check for proper spacing
        if re.search(r'\w\(\w', cpp_code):  # Functions with proper spacing
            score += 0.2
        
        # Check for const correctness
        if 'const' in cpp_code:
            score += 0.2
        
        # Check for RAII usage
        if 'std::unique_ptr' in cpp_code or 'std::shared_ptr' in cpp_code:
            score += 0.2
        
        # Check for proper includes
        if '#include' in cpp_code:
            score += 0.2
        
        return min(score, 1.0)
    
    @staticmethod
    def _check_error_handling(cpp_code: str) -> bool:
        """Check if code has proper error handling."""
        return 'try' in cpp_code and 'catch' in cpp_code
    
    @staticmethod
    def _check_modern_features(cpp_code: str) -> List[str]:
        """Check for modern C++ features."""
        features = []
        
        if 'auto' in cpp_code:
            features.append('auto')
        if 'std::unique_ptr' in cpp_code:
            features.append('smart_pointers')
        if 'std::vector' in cpp_code:
            features.append('stl_containers')
        if 'lambda' in cpp_code or '[]' in cpp_code:
            features.append('lambdas')
        if 'std::thread' in cpp_code:
            features.append('threading')
        
        return features

print("✅ Code Quality Analyzer implemented!")


## 4. Core Translation Logic

Now let's implement the main translation logic that coordinates all components.


In [ ]:
# Translation Comparison Data Class
@dataclass
class TranslationComparison:
    """Comparison of translations across different models."""
    model_results: Dict[str, TranslationResult]
    compilation_results: Dict[str, CompilationResult]
    execution_results: Dict[str, ExecutionResult]
    performance_metrics: Dict[str, PerformanceMetrics]
    quality_scores: Dict[str, Dict[str, Any]]
    best_model: Optional[str] = None
    comparison_summary: Optional[str] = None

# Main Code Translator
class CodeTranslator:
    """Main translator class that coordinates the entire translation process."""
    
    def __init__(self):
        self.llm_manager = LLMClientManager()
        self.available_models = self.llm_manager.get_available_models()
        
        if not self.available_models:
            print("⚠️ No LLM models available. Please check your API keys.")
    
    def translate_python_to_cpp(self, python_code: str, context: str = "", 
                               test_input: str = "", use_all_models: bool = True) -> TranslationComparison:
        """Translate Python code to C++ using available models."""
        
        if use_all_models:
            # Translate with all available models
            translation_results = self.llm_manager.translate_with_all_models(python_code, context)
        else:
            # Use first available model
            model_name = self.available_models[0]
            result = self.llm_manager.translate_with_model(model_name, python_code, context)
            translation_results = {model_name: result}
        
        # Compile and test each translation
        compilation_results = {}
        execution_results = {}
        performance_metrics = {}
        quality_scores = {}
        
        with CppCompiler() as compiler:
            for model_name, translation_result in translation_results.items():
                if not translation_result.success:
                    continue
                
                # Compile and test
                comp_result, exec_result = compiler.compile_and_test(
                    translation_result.translated_code, 
                    test_input
                )
                
                compilation_results[model_name] = comp_result
                if exec_result:
                    execution_results[model_name] = exec_result
                
                # Get performance metrics
                perf_metrics = self._get_performance_metrics(compiler, translation_result.translated_code, test_input)
                if perf_metrics:
                    performance_metrics[model_name] = perf_metrics
                
                # Analyze code quality
                quality_scores[model_name] = CodeQualityAnalyzer.analyze_cpp_quality(
                    translation_result.translated_code
                )
        
        # Determine best model
        best_model = self._determine_best_model(
            translation_results, compilation_results, execution_results, 
            performance_metrics, quality_scores
        )
        
        # Generate comparison summary
        comparison_summary = self._generate_comparison_summary(
            translation_results, compilation_results, execution_results,
            performance_metrics, quality_scores, best_model
        )
        
        return TranslationComparison(
            model_results=translation_results,
            compilation_results=compilation_results,
            execution_results=execution_results,
            performance_metrics=performance_metrics,
            quality_scores=quality_scores,
            best_model=best_model,
            comparison_summary=comparison_summary
        )
    
    def _get_performance_metrics(self, compiler: CppCompiler, cpp_code: str, test_input: str = "") -> Optional[PerformanceMetrics]:
        """Get comprehensive performance metrics."""
        compilation_result, execution_result = compiler.compile_and_test(cpp_code, test_input)
        
        if not compilation_result.success or not execution_result or not execution_result.success:
            return None
        
        # Get code size
        cpp_file = compiler._write_cpp_file(cpp_code)
        code_size = os.path.getsize(cpp_file)
        
        # Get executable size
        exe_size = 0
        if compilation_result.executable_path and os.path.exists(compilation_result.executable_path):
            exe_size = os.path.getsize(compilation_result.executable_path)
        
        return PerformanceMetrics(
            execution_time=execution_result.execution_time,
            memory_usage=execution_result.memory_usage,
            cpu_usage=0.0,  # Would need more complex monitoring
            code_size=code_size,
            compilation_time=compilation_result.compilation_time
        )
    
    def _determine_best_model(self, translation_results: Dict[str, TranslationResult],
                            compilation_results: Dict[str, CompilationResult],
                            execution_results: Dict[str, ExecutionResult],
                            performance_metrics: Dict[str, PerformanceMetrics],
                            quality_scores: Dict[str, Dict[str, Any]]) -> Optional[str]:
        """Determine the best model based on multiple criteria."""
        
        scores = {}
        
        for model_name in translation_results.keys():
            score = 0.0
            
            # Translation success (40% weight)
            if translation_results[model_name].success:
                score += 0.4
            
            # Compilation success (30% weight)
            if model_name in compilation_results and compilation_results[model_name].success:
                score += 0.3
            
            # Execution success (20% weight)
            if model_name in execution_results and execution_results[model_name].success:
                score += 0.2
            
            # Performance (5% weight)
            if model_name in performance_metrics:
                # Lower execution time is better
                exec_time = performance_metrics[model_name].execution_time
                if exec_time > 0:
                    score += 0.05 * (1.0 / (1.0 + exec_time))
            
            # Code quality (5% weight)
            if model_name in quality_scores:
                quality = quality_scores[model_name]
                style_score = quality.get('style_score', 0.0)
                score += 0.05 * style_score
            
            scores[model_name] = score
        
        if scores:
            return max(scores, key=scores.get)
        return None
    
    def _generate_comparison_summary(self, translation_results: Dict[str, TranslationResult],
                                   compilation_results: Dict[str, CompilationResult],
                                   execution_results: Dict[str, ExecutionResult],
                                   performance_metrics: Dict[str, PerformanceMetrics],
                                   quality_scores: Dict[str, Dict[str, Any]],
                                   best_model: Optional[str]) -> str:
        """Generate a summary of the comparison."""
        
        summary_parts = []
        
        # Overall success rates
        successful_translations = sum(1 for r in translation_results.values() if r.success)
        successful_compilations = sum(1 for r in compilation_results.values() if r.success)
        successful_executions = sum(1 for r in execution_results.values() if r.success)
        
        summary_parts.append(f"Translation Success: {successful_translations}/{len(translation_results)}")
        summary_parts.append(f"Compilation Success: {successful_compilations}/{len(compilation_results)}")
        summary_parts.append(f"Execution Success: {successful_executions}/{len(execution_results)}")
        
        # Best model
        if best_model:
            summary_parts.append(f"Best Model: {best_model}")
            
            # Best model details
            if best_model in performance_metrics:
                perf = performance_metrics[best_model]
                summary_parts.append(f"Best Model Performance:")
                summary_parts.append(f"  - Execution Time: {perf.execution_time:.4f}s")
                summary_parts.append(f"  - Memory Usage: {perf.memory_usage:.2f}MB")
                summary_parts.append(f"  - Compilation Time: {perf.compilation_time:.4f}s")
        
        # Quality comparison
        if quality_scores:
            summary_parts.append("Quality Scores:")
            for model, scores in quality_scores.items():
                summary_parts.append(f"  {model}:")
                summary_parts.append(f"    - Lines of Code: {scores.get('lines_of_code', 0)}")
                summary_parts.append(f"    - Comment Ratio: {scores.get('comment_ratio', 0):.2%}")
                summary_parts.append(f"    - Style Score: {scores.get('style_score', 0):.2f}")
                summary_parts.append(f"    - Complexity: {scores.get('complexity_score', 0)}")
        
        return "\n".join(summary_parts)

# Initialize the translator
translator = CodeTranslator()
print(f"✅ Code Translator initialized!")
print(f"Available models: {translator.available_models}")


## 5. Interactive Examples

Let's test the translator with some example Python code!


In [ ]:
# Example 1: Simple Fibonacci Function
python_code_1 = """
def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

def main():
    print("Fibonacci sequence:")
    for i in range(10):
        result = fibonacci(i)
        print(f"fibonacci({i}) = {result}")

if __name__ == "__main__":
    main()
"""

print("📝 Example 1: Fibonacci Function")
print("=" * 50)
print(python_code_1)


In [ ]:
# Test the translation
if translator.available_models:
    print("🔄 Translating Python code to C++...")
    print("This may take a few moments...")
    
    try:
        comparison = translator.translate_python_to_cpp(
            python_code_1, 
            "Fibonacci sequence generator",
            use_all_models=True
        )
        
        print(f"✅ Translation completed!")
        print(f"🏆 Best model: {comparison.best_model}")
        print(f"📊 Models used: {len(comparison.model_results)}")
        
        # Show results for each model
        for model_name, result in comparison.model_results.items():
            status = "✅ Success" if result.success else "❌ Failed"
            print(f"\n{model_name}: {status}")
            if result.success:
                print(f"  Translation time: {result.translation_time:.2f}s")
                if result.token_usage:
                    print(f"  Token usage: {result.token_usage}")
        
        # Show compilation results
        if comparison.compilation_results:
            print(f"\n🔨 Compilation Results:")
            for model_name, comp_result in comparison.compilation_results.items():
                status = "✅ Compiled" if comp_result.success else "❌ Failed"
                print(f"  {model_name}: {status}")
        
        # Show execution results
        if comparison.execution_results:
            print(f"\n⚡ Execution Results:")
            for model_name, exec_result in comparison.execution_results.items():
                status = "✅ Executed" if exec_result.success else "❌ Failed"
                print(f"  {model_name}: {status}")
                if exec_result.success and exec_result.output:
                    print(f"    Output: {exec_result.output.strip()}")
        
    except Exception as e:
        print(f"❌ Translation failed: {e}")
else:
    print("⚠️ No LLM models available. Please set your API keys.")


In [ ]:
# Display the best C++ code
if 'comparison' in locals() and comparison.best_model:
    best_result = comparison.model_results[comparison.best_model]
    print(f"🏆 Best C++ Code (from {comparison.best_model}):")
    print("=" * 60)
    print(best_result.translated_code)
    
    # Show quality metrics
    if comparison.best_model in comparison.quality_scores:
        quality = comparison.quality_scores[comparison.best_model]
        print(f"\n📊 Quality Metrics:")
        print(f"  Lines of code: {quality.get('lines_of_code', 0)}")
        print(f"  Comment ratio: {quality.get('comment_ratio', 0):.2%}")
        print(f"  Style score: {quality.get('style_score', 0):.2f}")
        print(f"  Complexity: {quality.get('complexity_score', 0)}")
        print(f"  Modern features: {quality.get('modern_cpp_features', [])}")
    
    # Show performance metrics
    if comparison.best_model in comparison.performance_metrics:
        perf = comparison.performance_metrics[comparison.best_model]
        print(f"\n⚡ Performance Metrics:")
        print(f"  Execution time: {perf.execution_time:.4f}s")
        print(f"  Memory usage: {perf.memory_usage:.2f}MB")
        print(f"  Compilation time: {perf.compilation_time:.4f}s")
        print(f"  Code size: {perf.code_size} bytes")


## 6. Additional Examples

Let's try a more complex example with classes and algorithms.


In [ ]:
# Example 2: Calculator Class
python_code_2 = """
class Calculator:
    def __init__(self):
        self.history = []
    
    def add(self, a, b):
        result = a + b
        self.history.append(f"{a} + {b} = {result}")
        return result
    
    def multiply(self, a, b):
        result = a * b
        self.history.append(f"{a} * {b} = {result}")
        return result
    
    def get_history(self):
        return self.history

def main():
    calc = Calculator()
    print("Calculator Demo")
    print(calc.add(5, 3))
    print(calc.multiply(4, 7))
    print("History:", calc.get_history())

if __name__ == "__main__":
    main()
"""

print("📝 Example 2: Calculator Class")
print("=" * 50)
print(python_code_2)


In [ ]:
# Test the second example
if translator.available_models:
    print("🔄 Translating Calculator class...")
    
    try:
        comparison2 = translator.translate_python_to_cpp(
            python_code_2, 
            "Calculator class with history tracking",
            use_all_models=True
        )
        
        print(f"✅ Translation completed!")
        print(f"🏆 Best model: {comparison2.best_model}")
        
        # Show summary
        print(f"\n📊 Summary:")
        print(comparison2.comparison_summary)
        
    except Exception as e:
        print(f"❌ Translation failed: {e}")
else:
    print("⚠️ No LLM models available.")


## 7. Summary and Results

This notebook demonstrates a comprehensive AI-powered code translation system that:

### Key Achievements:
- **Multi-LLM Support**: Successfully integrates OpenAI GPT, Anthropic Claude, and Google Gemini
- **C++ Compilation**: Automatically compiles and tests generated C++ code
- **Quality Analysis**: Provides detailed code quality metrics and performance benchmarking
- **Model Comparison**: Compares translation results across different AI models
- **Error Handling**: Robust error handling with detailed diagnostics

### Use Cases:
- **Learning C++**: Translate Python code to learn C++ equivalents
- **Code Migration**: Convert Python projects to C++ for performance
- **Educational Tool**: Compare different AI models' translation quality
- **Performance Analysis**: Benchmark Python vs C++ implementations

### Next Steps:
1. Set up your API keys for OpenAI, Anthropic, and Google
2. Run the notebook cells to test the translation system
3. Experiment with your own Python code
4. Compare results across different AI models
5. Analyze code quality and performance metrics

**Happy coding! 🎉**


rom 